# 2a. DATA TRANSFORMATION & STANDARDIZATION

This notebook focuses on data cleaning operations:
1. **Text Normalization** - Convert to lowercase, fix encoding issues
2. **Typo Detection & Correction** - Fix special characters (caffÿ → caffè)
3. **Column Renaming** - Clean up problematic column names
4. **Data Wrangling** - Create macro-categories for exercise types

## 2a.1 Imports and Load Data

In [59]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 140)

In [60]:
# Load the original dataset
MILANO = pd.read_csv("Comune-di-Milano-Pubblici-esercizi(in)-2.csv", sep=";", encoding="utf-8")
print(f"Original shape: {MILANO.shape}")
MILANO.head()

Original shape: (6904, 13)


,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
0,NaN,NaN,ALZ NAVIGLIO GRANDE N. 12 ; isolato:057; (z.d. 6),ALZ,NAVIGLIO GRANDE,12,5144,6,NaN,NaN,NaN,"Ristorante, trattoria, osteria;Genere Merceol....",83.0
1,NaN,NaN,ALZ NAVIGLIO GRANDE N. 44 (z.d. 6),ALZ,NAVIGLIO GRANDE,44,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,26.0
2,NaN,NaN,ALZ NAVIGLIO GRANDE N. 48 (z.d. 6),ALZ,NAVIGLIO GRANDE,48,5144,6,NaN,NaN,NaN,Bar gastronomici e simili,58.0
3,NaN,NaN,ALZ NAVIGLIO GRANDE N. 8 (z.d. 6),ALZ,NAVIGLIO GRANDE,8,5144,6,NaN,NaN,NaN,"BAR CAFFÿý E SIMILI;Ristorante, trattoria, ost...",101.0
4,NaN,NaN,ALZ NAVIGLIO PAVESE N. 24 (z.d. 6),ALZ,NAVIGLIO PAVESE,24,5161,6,NaN,NaN,NaN,Bar gastronomici e simili,51.0


---
# 1. TEXT NORMALIZATION

## 1.1 Convert Text Columns to Lowercase

In [61]:
# Select text columns
text_cols = MILANO.select_dtypes(include="object").columns
print(f"Text columns: {list(text_cols)}")

Text columns: ['þÿTipo esercizio storico pe', 'Insegna', 'Ubicazione', 'Tipo via', 'Descrizione via', 'Civico', 'Forma commercio', 'Forma commercio prev', 'Forma vendita', 'Settore storico pe']


In [62]:
# Convert to lowercase
MILANO[text_cols] = MILANO[text_cols].apply(lambda col: col.str.lower())
MILANO.head()

,þÿTipo esercizio storico pe,Insegna,Ubicazione,Tipo via,Descrizione via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione
0,NaN,NaN,alz naviglio grande n. 12 ; isolato:057; (z.d. 6),alz,naviglio grande,12,5144,6,NaN,NaN,NaN,"ristorante, trattoria, osteria;genere merceol....",83.0
1,NaN,NaN,alz naviglio grande n. 44 (z.d. 6),alz,naviglio grande,44,5144,6,NaN,NaN,NaN,bar gastronomici e simili,26.0
2,NaN,NaN,alz naviglio grande n. 48 (z.d. 6),alz,naviglio grande,48,5144,6,NaN,NaN,NaN,bar gastronomici e simili,58.0
3,NaN,NaN,alz naviglio grande n. 8 (z.d. 6),alz,naviglio grande,8,5144,6,NaN,NaN,NaN,"bar caffÿý e simili;ristorante, trattoria, ost...",101.0
4,NaN,NaN,alz naviglio pavese n. 24 (z.d. 6),alz,naviglio pavese,24,5161,6,NaN,NaN,NaN,bar gastronomici e simili,51.0


## 1.2 Fix Column Names

In [63]:
# The column 'þÿTipo esercizio storico pe' has encoding issues
print("Original columns:")
print(MILANO.columns.tolist())

Original columns:
['þÿTipo esercizio storico pe', 'Insegna', 'Ubicazione', 'Tipo via', 'Descrizione via', 'Civico', 'Codice via', 'ZD', 'Forma commercio', 'Forma commercio prev', 'Forma vendita', 'Settore storico pe', 'Superficie somministrazione']


In [64]:
# Rename problematic columns
MILANO = MILANO.rename(columns={
    "þÿTipo esercizio storico pe": "Tipo esercizio storico pubblico esercizio",
    "Ubicazione": "Indirizzo",
    "Descrizione via": "Nome via",
    "Forma commercio prev": "Forma commercio precedente",
    "Settore storico pe": "Settore storico pubblico esercizio"
})

print("Renamed columns:")
print(MILANO.columns.tolist())

Renamed columns:
['Tipo esercizio storico pubblico esercizio', 'Insegna', 'Indirizzo', 'Tipo via', 'Nome via', 'Civico', 'Codice via', 'ZD', 'Forma commercio', 'Forma commercio precedente', 'Forma vendita', 'Settore storico pubblico esercizio', 'Superficie somministrazione']


---
# 2. TYPO DETECTION & CORRECTION

## 2.1 Detect Non-ASCII Characters

In [65]:
# Find rows with problematic characters (ÿ, ý)
chars = ["ÿ", "ý"]
pattern = "|".join(map(lambda x: "\\" + x, chars))

mask = MILANO.select_dtypes(include="object").apply(
    lambda col: col.astype(str).str.contains(pattern, na=False)
).any(axis=1)

df_bad = MILANO[mask]
print(f"Rows with problematic characters: {len(df_bad)}")
df_bad.head()

Rows with problematic characters: 5243


,Tipo esercizio storico pubblico esercizio,Insegna,Indirizzo,Tipo via,Nome via,Civico,Codice via,ZD,Forma commercio,Forma commercio precedente,Forma vendita,Settore storico pubblico esercizio,Superficie somministrazione
3,NaN,NaN,alz naviglio grande n. 8 (z.d. 6),alz,naviglio grande,8,5144,6,NaN,NaN,NaN,"bar caffÿý e simili;ristorante, trattoria, ost...",101.0
6,NaN,NaN,bst di porta nuova n. 10 con ingr.su piazza xx...,bst,di porta nuova,10,1062,1,NaN,NaN,NaN,bar gastronomici e simili;tavola fredda;genere...,341.0
7,NaN,NaN,bst di porta volta n. 9 (z.d. 1),bst,di porta volta,9,1066,1,NaN,NaN,NaN,genere merceol.autorizz.sanit.;bar gastronomic...,51.0
8,NaN,NaN,bst di porta volta n. 9 (z.d. 1),bst,di porta volta,9,1066,1,NaN,NaN,NaN,"wine,birr.,pub enot.,caff.,the;bar caffÿý;gene...",26.0
9,NaN,NaN,bst di porta volta num.018/a ; (z.d. 1),bst,di porta volta,NaN,1066,1,NaN,NaN,NaN,"bar caffÿý e simili;ristorante, trattoria, ost...",115.0


## 2.2 Fix caffÿ → caffè

In [66]:
# Count occurrences of 'caffÿ' pattern
count = 0
for c in MILANO.select_dtypes(include="object").columns:
    count += MILANO[c].astype(str).str.count(r"\bcaff[ÿý]").sum()

print(f"Occurrences of 'caffÿ/caffý' pattern: {count}")

Occurrences of 'caffÿ/caffý' pattern: 10770


In [67]:
# Replace caffÿ/caffý with caffè
text_cols = MILANO.select_dtypes(include="object").columns
MILANO[text_cols] = MILANO[text_cols].apply(
    lambda col: col.str.replace(r"\bcaff[ÿý]", "caffè", regex=True)
)

# Also fix caffèý pattern
MILANO[text_cols] = MILANO[text_cols].apply(
    lambda col: col.str.replace(r"\bcaffè[ý]", "caffè", regex=True)
)

print("Fixed caffè pattern")

Fixed caffè pattern


In [68]:
# Remove remaining stray ÿ/ý characters from Indirizzo and Insegna
for col in ["Indirizzo", "Insegna"]:
    if col in MILANO.columns:
        MILANO[col] = MILANO[col].astype(str).str.replace("[ÿý]", "", regex=True)

print("Removed stray special characters")

Removed stray special characters


In [69]:
# Verify no more problematic characters remain
mask = MILANO.select_dtypes(include="object").apply(
    lambda col: col.astype(str).str.contains(pattern, na=False)
).any(axis=1)

print(f"Remaining rows with problematic characters: {mask.sum()}")

Remaining rows with problematic characters: 0


## 2.3 Check for Other Non-ASCII Characters

In [70]:
# Pattern for non-ASCII excluding common Italian accented letters
pattern_non_ascii = r"[^\x00-\x7FàèéìòùÀÈÉÌÒÙ]"

mask = MILANO.select_dtypes(include="object").apply(
    lambda col: col.astype(str).str.contains(pattern_non_ascii, na=False)
).any(axis=1)

print(f"Rows with other non-ASCII characters: {mask.sum()}")
if mask.sum() > 0:
    display(MILANO[mask].head())

Rows with other non-ASCII characters: 0


---
# 3. DATA WRANGLING: CREATE MACRO-CATEGORIES

Create simplified macro-categories for exercise types based on the messy `Settore storico pe` column.

In [79]:
# Define column names
settore_col = "Settore storico pubblico esercizio"
tipo_col = "Tipo esercizio storico pubblico esercizio"

# Normalize sector text
MILANO["settore_norm"] = MILANO[settore_col].astype(str).str.upper().fillna("")

In [ ]:
# Define patterns for each macro-category

# BAR
patterns_bar = [
    "BAR CAFFE", "CAFF", "BAR GASTRONOM", "BIRRERIA", "SALE DA BALLO",
    "BAR", "DISCO", "LOCALI NOTTURNI", "SPACCIO BEVANDE ANALCOLICHE",
    "GIOC", "SOMMINISTRAZIONE", "WINE", "PUB"
]

# PIZZERIA
patterns_piz = ["PIZZERIA", "PIZZERIE E SIMILI"]

# RISTORANTE
patterns_rist = ["RISTORA", "OSTERIA", "CUCINA", "TRATTORIA"]

# GASTRONOMIA
patterns_gast = [
    "GENERE MERCEOL", "PRODOTTI DI GASTRONOMIA", "PROD DI GASTRO",
    "TAVOLA FREDDA", "CIBI COTTI", "CIBI COTTI PRECONFEZIONATI",
    "MENSA", "TAVOLA CALDA", "TAV.CALDE,SELF SERVICE,FAST F",
    "SELF SERVICE", "FAST F"
]

# GELATERIA
patterns_gel = ["BAR PASTIC", "GELATERIA"]

In [ ]:
def build_mask(patterns):
    """Return boolean mask for rows containing any of the patterns."""
    regex = "|".join(patterns)
    return MILANO["settore_norm"].str.contains(regex, na=False)

mask_bar = build_mask(patterns_bar)
mask_piz = build_mask(patterns_piz)
mask_rist = build_mask(patterns_rist)
mask_gast = build_mask(patterns_gast)
mask_gel = build_mask(patterns_gel)

In [ ]:
# Assign macro-categories with hierarchy (RISTORANTE > PIZZERIA > BAR > GASTRONOMIA > GELATERIA)
MILANO["Tipo_macro"] = "ALTRO"

# Priority from lowest to highest
MILANO.loc[mask_gel, "Tipo_macro"] = "GELATERIA"
MILANO.loc[mask_gast, "Tipo_macro"] = "GASTRONOMIA"
MILANO.loc[mask_bar, "Tipo_macro"] = "BAR"
MILANO.loc[mask_piz, "Tipo_macro"] = "PIZZERIA"
MILANO.loc[mask_rist, "Tipo_macro"] = "RISTORANTE"  # Highest priority

# Check distribution
MILANO["Tipo_macro"].value_counts()

Tipo_macro
BAR            3654
RISTORANTE     2734
GASTRONOMIA     246
PIZZERIA        246
ALTRO            21
GELATERIA         3
Name: count, dtype: int64

In [ ]:
# Clean up temporary column
MILANO = MILANO.drop(columns=["settore_norm"])
print("Created 'Tipo_macro' column with simplified categories")

Created 'Tipo_macro' column with simplified categories


---
# 4. SAVE TRANSFORMED DATASET

In [ ]:
# Summary of transformations applied
print("=== TRANSFORMATIONS APPLIED ===")
print("1. Converted all text columns to lowercase")
print("2. Renamed problematic column names")
print("3. Fixed caffÿ/caffý → caffè typos")
print("4. Removed stray special characters")
print("5. Created Tipo_macro simplified categories")
print(f"\nFinal shape: {MILANO.shape}")

=== TRANSFORMATIONS APPLIED ===
1. Converted all text columns to lowercase
2. Renamed problematic column names
3. Fixed caffÿ/caffý → caffè typos
4. Removed stray special characters
5. Repaired inconsistent Indirizzo field
6. Filled missing Civico from Indirizzo
7. Created Tipo_macro simplified categories

Final shape: (6904, 14)


In [ ]:
# Save the transformed dataset
MILANO.to_csv("MILANO_transformed.csv", index=False, sep=";")
print("Saved: MILANO_transformed.csv")

Saved: MILANO_transformed.csv


In [ ]:
# Preview final dataset
MILANO.head()

,Tipo esercizio storico pubblico esercizio,Insegna,Indirizzo,Tipo via,Nome via,Civico,Codice via,ZD,Forma commercio,Forma commercio prev,Forma vendita,Settore storico pe,Superficie somministrazione,Tipo_macro
0,NaN,nan,alz naviglio grande n. 12 ; isolato:057; (z.d. 6),alz,naviglio grande,12,5144,6,NaN,NaN,NaN,"ristorante, trattoria, osteria;genere merceol....",83.0,RISTORANTE
1,NaN,nan,alz naviglio grande n. 44 (z.d. 6),alz,naviglio grande,44,5144,6,NaN,NaN,NaN,bar gastronomici e simili,26.0,BAR
2,NaN,nan,alz naviglio grande n. 48 (z.d. 6),alz,naviglio grande,48,5144,6,NaN,NaN,NaN,bar gastronomici e simili,58.0,BAR
3,NaN,nan,alz naviglio grande n. 8 (z.d. 6),alz,naviglio grande,8,5144,6,NaN,NaN,NaN,"bar caffè e simili;ristorante, trattoria, osteria",101.0,RISTORANTE
4,NaN,nan,alz naviglio pavese n. 24 (z.d. 6),alz,naviglio pavese,24,5161,6,NaN,NaN,NaN,bar gastronomici e simili,51.0,BAR
